# Polars Quick Start

- [User Guide](https://pola-rs.github.io/polars-book/user-guide/index.html)
- [API Reference](https://pola-rs.github.io/polars/py-polars/html/reference/functions.html)

## Polars DataFrame

Polars 的核心 API 和 Pandas 类似, 也是 DataFrame. 与 Pandas 相比有这么几点不同:

1. Polars 没有 index, 所有的 row 只有从 0, 1 开始的自然 index.
2. 在内存中 Polars 使用基于连续内存的 Arrow arrays, 而 Pandas 使用的是 Numpy Array. 简单来说 Arrow 是更新的标准, 读取数据更快, 内存更紧凑, 使用内存更少.
3. Polars 对利用多个 CPU 进行并行计算的支持比 Pandas 更好.
4. Polars 支持 Lazy Evaluation, 也就是你对数据的操作会先被标记而不是真正被执行, 当你定义好了一系列操作以后, 最后调用会触发 Evaluation 的 API (例如 collect) 时计算才会被真正执行. 这样可以对操作进行预优化.

参考资料:

- [Coming from pandas](https://pola-rs.github.io/polars-book/user-guide/coming_from_pandas.html)


In [2]:
import polars as pl

In [3]:
pl.__version__

'0.13.59'

## Create DataFrame

In [6]:
df = pl.DataFrame({
    "id": ["a", "b", "c"],
    "value": [1, 2, 3],
})
df

id,value
str,i64
"""a""",1
"""b""",2
"""c""",3


In [10]:
df = pl.DataFrame([
    {"id": "i", "value": 1},
    {"id": "j", "value": 2},
    {"id": "k", "value": 3},
])
df

id,value
str,i64
"""i""",1
"""j""",2
"""k""",3


In [11]:
df = pl.DataFrame(
    [
        ("x", 1),
        ("y", 2),
        ("z", 3),
    ],
    columns=["id", "value"],
)
df

id,value
str,i64
"""x""",1
"""y""",2
"""z""",3


## Iterate on Rows

Polars 是列式存储. 虽然它支持根据行遍历, 但是这样无法利用多核并行以及优化. 可以做但是不推荐.

Reference:

- [List context and Row wise computations](https://pola-rs.github.io/polars-book/user-guide/dsl/list_context.html)

In [25]:
# 用 df.rows() 方法返回 list of tuple
for row in df.rows():
    print(row)

('x', 1)
('y', 2)
('z', 3)


In [27]:
# 用 df.row() 方法根据 index 访问 row tuple
df.row(1)

('y', 2)

## Iterator on Column

在 Polars 中 Column 的数据结构叫做 Series, 代表的是一系列数据类型一致的数组, 且没有 index. Pandas 中也有 Series 的概念, 不过 Pandas 中的 Series 类似于 Dictionary, 是一个 key, value pair, 也并没有要求 value 的数据类型需要一致.

In [28]:
df.columns

['id', 'value']

In [29]:
df["id"]

shape: (3,)
Series: 'id' [str]
[
	"x"
	"y"
	"z"
]

In [30]:
df["id"].to_list()

['x', 'y', 'z']

In [31]:
for id in df["id"]:
    print(id)

x
y
z


In [33]:
for series in df:
    print(series)

shape: (3,)
Series: 'id' [str]
[
	"x"
	"y"
	"z"
]
shape: (3,)
Series: 'value' [i64]
[
	1
	2
	3
]
